In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import os

# Configurações
model_name = "BAAI/bge-m3"
output_folder = 'vectorstore'
existing_db_path = 'vectorstore/db_faiss_all_pdfs'


def load_and_process_pdfs(pdf_folder):
    documents = []
    for filename in os.listdir(pdf_folder):
        if filename.endswith('.pdf'):
            file_path = os.path.join(pdf_folder, filename)
            loader = PyPDFLoader(file_path)
            documents.extend(loader.load())
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    split_docs = text_splitter.split_documents(documents)
    return split_docs


def get_vectorstore(pdf_folder):
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    if os.path.exists(existing_db_path):
        return FAISS.load_local(existing_db_path, embeddings, allow_dangerous_deserialization=True)
    else:
        docs = load_and_process_pdfs(pdf_folder)
        vectorstore = FAISS.from_documents(docs, embeddings)
        vectorstore.save_local(existing_db_path)
        return vectorstore


vectorstore = get_vectorstore('pdf_ppg')
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={'k': 5, 'fetch_k': 6})

c:\Users\niels\anaconda3\envs\LLM\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
def model_ollama(model="llama3.2", temperature=0.1):
    return ChatOllama(model=model, temperature=temperature)

llm = model_ollama()

In [4]:
context_q_system_prompt = """
Dado o histórico de chat e a pergunta de acompanhamento, que pode fazer referência ao contexto no histórico do chat, 
formule uma pergunta independente que possa ser entendida sem o histórico do chat. 
NÃO responda à pergunta, apenas reformule-a se necessário e, caso contrário, retorne-a como está.
"""

qa_prompt_template = """
Você é um assistente virtual especializado em analisar e responder perguntas sobre documentos.
Use os seguintes trechos de contexto recuperado para responder à pergunta.
Se você não souber a resposta, diga honestamente que não sabe. Mantenha a resposta concisa e focada.
Se for solicitado a listar as referências dos artigos, informações específicas do documento, faça-o de forma estruturada e clara.
Responda em português.

Pergunta: {input}

Contexto: {context}

Resposta:
"""

context_q_prompt = ChatPromptTemplate.from_messages([
    ("system", context_q_system_prompt),
    ("human", "Pergunta: {input}")
])

history_aware_retriever = create_history_aware_retriever(llm=llm, retriever=retriever, prompt=context_q_prompt)

qa_prompt = PromptTemplate.from_template(qa_prompt_template)
qa_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

# Função para fazer perguntas
def ask_question(question, chat_history=''): 
    result = rag_chain.invoke({"input": question, "chat_history": chat_history})
    return result['answer']

In [8]:
user_query = 'Liste todos os desafios?'
response = ask_question(user_query)
print(response)

Aqui estão os desafios listados:

**DESAFIOS SOFT TECH**

1. Caracterização do envelope operacional de poços desconectados e em monitoramento via sistema hidro acústico
2. Desenvolver um software ágil e simples que represente graficamente todas as barreiras de um poço, englobando também a estrutura montada na sonda em um esquema educativo (RIG PROCESS VIEW)
3. DESAFIO 3 - Não foi especificado

**DESAFIOS DEEP TECH**

4. Sistema para monitoração de perda de espessura em grandes trechos de tubulação ou dutos, com comunicação remota de dados
5. DESAFIO 5 - Não foi especificado
6. DESAFIO 6 - Não foi especificado
7. DESAFIO 7 - Não foi especificado
8. DESAFIO 8 - Não foi especificado
9. DESAFIO 9 - Não foi especificado
10. Sistema para monitoração de perda de espessura em grandes trechos de tubulação ou dutos, com comunicação remota de dados (DESAFIOS DEEP TECH)
11. DESAFIO 11 - Não foi especificado

Observações:

* Desafio 3 não foi especificado.
* Os desafios 5 a 11 não foram especificad

In [11]:
user_query = """Sou cientista de dados, especialista em engenharia de dados e visão computacional. 
Qual seria os desafios baseado nas minhas especialidades? """
response = ask_question(user_query)
print(response)

Com base nos desafios apresentados, posso identificar alguns dos principais desafios que você pode enfrentar como cientista de dados, especialista em engenharia de dados e visão computacional:

1. **Desafio 3 - SOFT TECH**: Estruturação e integração de dados para visualização e análises de integridade em sistemas especialistas para dutos rígidos submarinos.
 * Desafios:
 + Tratamento de dados não estruturados e descentralizados
 + Integração com sistemas especialistas como Integridutos e IntegriSpan
 + Visualização eficaz dos dados para análises de integridade
2. **Desafio 10 - DEEP TECH**: Sistema para monitoração de perda de espessura em grandes trechos de tubulação ou dutos, com comunicação remota de dados.
 * Desafios:
 + Monitoramento de longas distâncias com tecnologias digitais e satélites
 + Coleta e comunicação de dados remotos de forma eficiente
 + Redução do tempo de resposta em caso de problema de integridade
3. **Desafio 2 - SOFT TECH**: Desenvolver um software ágil e simp

In [12]:
user_query = 'Quais os Desafios que envolvem processamento de imagem ou utilização de drone ?'
response = ask_question(user_query)
print(response)

Olá! Estou aqui para ajudar a analisar e responder às suas perguntas sobre documentos relacionados a processamento de imagem e utilização de drones.

Para responder à sua pergunta, gostaria de destacar alguns desafios que envolvem o processamento de imagem ou a utilização de drones:

1. **Qualidade da imagem**: A qualidade da imagem capturada por câmeras de drone ou sensores pode ser limitada pela distância e pelo ambiente em que está sendo capturada.
2. **Processamento de imagem**: O processamento de imagem é um desafio, especialmente quando se trata de imagens de alta resolução e complexidade.
3. **Análise de dados**: A análise de dados gerados pelas imagens pode ser um desafio, especialmente quando se trata de grandes conjuntos de dados.
4. **Segurança**: A segurança é um desafio, especialmente quando se trata de drones que podem estar em áreas perigosas ou com acesso limitado.

Esses são apenas alguns dos desafios que envolvem o processamento de imagem e a utilização de drones. Se 